# HOMEWORK 10

results of each tracking are in video files

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt


In [39]:
cap = cv2.VideoCapture('videomoving.mp4')

# take first frame of the video
ret,frame = cap.read()

# Get the height and width of the frame (required to be an interger)
width = int(cap.get(3)) 
height = int(cap.get(4))

# Define the codec and create VideoWriter object. The output is stored in '*.avi' file.
out = cv2.VideoWriter('tank_mean_shift.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

# setup initial location of window
r,h,c,w = 300,60,650,60  # simply hardcoded the values
track_window = (c,r,w,h)

# set up the ROI for tracking
roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv_roi, np.array((0., 60.,32.)), np.array((180.,255.,255.)))
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while(1):
    ret, frame = cap.read() 

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)

        # apply meanshift to get the new location
        ret, track_window = cv2.meanShift(dst, track_window, term_crit)

        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), (255,255,255),2)
        out.write(img2)
        

    else:
        break

cap.release()
out.release()

I first tried something more simple. This doesn't work because this method requires more contrast objects. It's clear to see that it tries to stick on white car. But fails on tank

In [36]:
def create_tracker(tracker_type):
    if tracker_type == 'MIL':
        return cv2.TrackerMIL_create()
    elif tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    elif tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    else:
        raise ValueError("Incorrect tracker type")

cap = cv2.VideoCapture('videomoving.mp4')  # Replace with your video file path

tracker_type = 'KCF'

# Example initial bounding boxes for objects (x, y, w, h)
initial_bboxes = [(640,300,80,50)]  # Replace with your bounding boxes

trackers = []
ret, frame = cap.read()

width = int(cap.get(3)) 
height = int(cap.get(4))

for bbox in initial_bboxes:
    tracker = create_tracker(tracker_type)
    tracker.init(frame, bbox)
    trackers.append(tracker)

out = cv2.VideoWriter('tank_kcf.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for tracker in trackers:
        success, bbox = tracker.update(frame)
        #print(success, bbox)
        x, y, w, h = map(int, bbox)
        img2 = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        out.write(img2)

    #cv2.imshow('Frame', frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()



KCF works better, but fails to work  on frame when street lamp cross the tank.

In [37]:
def create_tracker(tracker_type):
    if tracker_type == 'MIL':
        return cv2.TrackerMIL_create()
    elif tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    elif tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    else:
        raise ValueError("Incorrect tracker type")

cap = cv2.VideoCapture('videomoving.mp4')  # Replace with your video file path

tracker_type = 'MIL'

# Example initial bounding boxes for objects (x, y, w, h)
initial_bboxes = [(640,300,80,50)]  # Replace with your bounding boxes

trackers = []
ret, frame = cap.read()

width = int(cap.get(3)) 
height = int(cap.get(4))

for bbox in initial_bboxes:
    tracker = create_tracker(tracker_type)
    tracker.init(frame, bbox)
    trackers.append(tracker)

out = cv2.VideoWriter('tank_mil.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for tracker in trackers:
        success, bbox = tracker.update(frame)
        #print(success, bbox)
        x, y, w, h = map(int, bbox)
        img2 = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        out.write(img2)

    #cv2.imshow('Frame', frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()



MIL works more better and successfully passed place where KCF fails, but failed in next frames

In [38]:
def create_tracker(tracker_type):
    if tracker_type == 'MIL':
        return cv2.TrackerMIL_create()
    elif tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    elif tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    else:
        raise ValueError("Incorrect tracker type")

cap = cv2.VideoCapture('videomoving.mp4')  # Replace with your video file path

tracker_type = 'CSRT'

# Example initial bounding boxes for objects (x, y, w, h)
initial_bboxes = [(640,300,80,50)]  # Replace with your bounding boxes

trackers = []
ret, frame = cap.read()

width = int(cap.get(3)) 
height = int(cap.get(4))

for bbox in initial_bboxes:
    tracker = create_tracker(tracker_type)
    tracker.init(frame, bbox)
    trackers.append(tracker)

out = cv2.VideoWriter('tank_csrt.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    for tracker in trackers:
        success, bbox = tracker.update(frame)
        #print(success, bbox)
        x, y, w, h = map(int, bbox)
        img2 = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        out.write(img2)

    #cv2.imshow('Frame', frame)

    if cv2.waitKey(30) & 0xFF == 27:
        break

cap.release()
out.release()



CSRT wins in my case and no surprise because it's more robust. Strange that speed a bit better than MIL

In [47]:
cap = cv2.VideoCapture('videomoving.mp4')

# Get the height and width of the frame (required to be an interger)
width = int(cap.get(3)) 
height = int(cap.get(4))

# Define the codec and create VideoWriter object. The output is stored in '*.avi' file.
out = cv2.VideoWriter('optical_flow.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 30, (width, height))

# Set parameters for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Set parameters for lucas kanade optical flow
lucas_kanade_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
# Used to create our trails for object movement in the image 
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Define ROI (x, y, width, height)
roi_x, roi_y, roi_w, roi_h = 640,300,80,50  # Replace with your desired values
mask = np.zeros_like(prev_gray)
mask[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w] = 255


# Find inital corner locations
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask = mask, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(prev_frame)

while(1):
    ret, frame = cap.read()

    if ret == True:
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      # calculate optical flow
      new_corners, status, errors = cv2.calcOpticalFlowPyrLK(prev_gray, 
                                                            frame_gray, 
                                                            prev_corners, 
                                                            None, 
                                                            **lucas_kanade_params)

      # Select and store good points
      good_new = new_corners[status==1]
      good_old = prev_corners[status==1]

      # Draw the tracks
      for i,(new,old) in enumerate(zip(good_new, good_old)):
          a, b = new.ravel()
          c, d = old.ravel()
          a, b, c, d = int(a), int(b), int(c), int(d) 
          mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
          frame = cv2.circle(frame, (a,b), 5, color[i].tolist(),-1)
          
      img = cv2.add(frame,mask)

      # Save Video
      out.write(img)
      # Show Optical Flow
      #imshow('Optical Flow - Lucas-Kanade',img)

      # Now update the previous frame and previous points
      prev_gray = frame_gray.copy()
      prev_corners = good_new.reshape(-1,1,2)

    else:
      break
    
cap.release()
out.release()

[[[699. 327.]]

 [[661. 325.]]

 [[670. 325.]]

 [[680. 327.]]]


tried also optical flow. Results are not bad in fact that this method is much faster. We can draw also box around motion point to have similar results as in tracking, but thsis method also failed when tank crossed street light. But I like how fast this method